In [1]:
import duckdb

In [29]:
cn = duckdb.connect()

cn.execute("""
    create or replace view v_order_header as select * from read_csv_auto('~/test_data/orders_header*.csv');
    create or replace view v_order_detail as select * from read_csv_auto('~/test_data/orders_detail*.csv');  
    create or replace view v_inventory as select * from read_csv('~/test_data/inventory*.csv',header=true);
""")

In [30]:
cn.sql("from v_order_header limit 1").show()
cn.sql("from v_order_detail limit 1").show()
cn.sql("from v_inventory limit 2").show()

┌──────────┬─────────────┬───────────────────────────────┬─────────┬──────────┬─────────┬────────────────┬──────────┬─────────────┐
│ order_id │ customer_id │          order_date           │ status  │ store_id │ channel │ payment_method │ ship_zip │ total_cents │
│  int64   │    int64    │   timestamp with time zone    │ varchar │  int64   │ varchar │    varchar     │ varchar  │    int64    │
├──────────┼─────────────┼───────────────────────────────┼─────────┼──────────┼─────────┼────────────────┼──────────┼─────────────┤
│        0 │     4887431 │ 2026-01-13 20:36:57.017895-05 │ CLOSE   │     4814 │ Mobile  │ Card           │ 97085    │      137504 │
└──────────┴─────────────┴───────────────────────────────┴─────────┴──────────┴─────────┴────────────────┴──────────┴─────────────┘

┌──────────┬─────────┬───────────────┬───────┬──────────────────┬──────────────────┐
│ order_id │ line_id │      sku      │  qty  │ unit_price_cents │ line_total_cents │
│  int64   │  int64  │    varchar    

In [26]:
sql = """
    select iv.category
        ,count(distinct oh.order_id) as num_orders
        ,count(distinct od.line_id) as num_order_lines
        ,sum(od.qty) as total_units_sold
        ,sum(od.line_total_cents)/100.0 as total_sales_dollars
    from v_order_header as oh
        inner join v_order_detail as od on oh.order_id = od.order_id
        inner join v_inventory as iv on od.sku = iv.sku
    group by all
    limit 5
"""
cn.sql(sql).show()

┌──────────┬────────────┬─────────────────┬──────────────────┬─────────────────────┐
│ category │ num_orders │ num_order_lines │ total_units_sold │ total_sales_dollars │
│ varchar  │   int64    │      int64      │      int128      │       double        │
├──────────┼────────────┼─────────────────┼──────────────────┼─────────────────────┤
│ Fishing  │   14416978 │              10 │         61362885 │      15492854569.24 │
│ Footwear │   14448650 │              10 │         61550709 │      15543029663.03 │
│ Hunting  │   14430691 │              10 │         61464318 │      15521381265.29 │
│ Boating  │   14442148 │              10 │         61527188 │      15536195115.51 │
│ Camping  │   14438070 │              10 │         61511574 │       15531742537.1 │
└──────────┴────────────┴─────────────────┴──────────────────┴─────────────────────┘



In [31]:
cn.sql("""
        select order_id
       from v_order_header oh
       group by 1
       having count(*)>=2   
""").show()

cn.sql("""select order_id, line_id
       from v_order_detail
       group by all
       having count(*)>=2
       limit 5
""").show()

cn.sql("""
    select sku
    from v_inventory
    group by all
    having count(*)>=2
    limit 5
""").show()

┌──────────┐
│ order_id │
│  int64   │
├──────────┤
│  0 rows  │
└──────────┘

┌──────────┬─────────┐
│ order_id │ line_id │
│  int64   │  int64  │
├──────────┴─────────┤
│       0 rows       │
└────────────────────┘

┌─────────┐
│   sku   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘



In [32]:
sql = """
    select 'order header' as source, count(*) as cnt from v_order_header
    union all
    select 'order detail' as source, count(*) as cnt from v_order_detail
    union all
    select 'inventory' as source, count(*) as cnt from v_inventory
"""
cn.sql(sql).show()

┌──────────────┬───────────┐
│    source    │    cnt    │
│   varchar    │   int64   │
├──────────────┼───────────┤
│ order header │  29864729 │
│ order detail │ 164272472 │
│ inventory    │   4967925 │
└──────────────┴───────────┘



In [33]:
sql = f"""
    select oh.channel
        , iv.location
        , sum(od.qty) as tot_qty
    from v_order_header oh
        join v_order_detail od on oh.order_id = od.order_id
        join v_inventory iv on od.sku = iv.sku
    group by all
    limit 5
"""
cn.sql(sql).show()

┌─────────┬─────────────────┬─────────┐
│ channel │    location     │ tot_qty │
│ varchar │     varchar     │ int128  │
├─────────┼─────────────────┼─────────┤
│ Store   │ Aisle-33-Bin-19 │  133360 │
│ Store   │ Aisle-14-Bin-9  │  134208 │
│ Online  │ Aisle-10-Bin-4  │  131813 │
│ Online  │ Aisle-39-Bin-11 │  131400 │
│ Online  │ Aisle-46-Bin-1  │  127839 │
└─────────┴─────────────────┴─────────┘



In [25]:
sql = """
    select oh.payment_method
        ,sum(od.qty) as tot_qty
        ,sum(od.line_total_cents)/100.0 as tot_sales_dollars
    from v_order_header oh
        left join v_order_detail od on oh.order_id = od.order_id
    group by all
    limit 5
"""
cn.sql(sql).show()

┌────────────────┬──────────┬───────────────────┐
│ payment_method │ tot_qty  │ tot_sales_dollars │
│    varchar     │  int128  │      double       │
├────────────────┼──────────┼───────────────────┤
│ Cash           │ 98432381 │     24854426644.1 │
│ ApplePay       │ 98465878 │    24865786210.41 │
│ PayPal         │ 98434248 │    24855649021.85 │
│ Gift           │ 98437515 │    24852533071.68 │
│ Card           │ 98391623 │    24844961406.79 │
└────────────────┴──────────┴───────────────────┘



In [3]:
cn2 = duckdb.connect('~/test_dbs/duckland_local.db')
cn2.sql("select * from ord_stats_by_category limit 5").show()
cn2.close()

┌──────────┬────────────┬─────────────────┬──────────────────┬─────────────────────┐
│ category │ num_orders │ num_order_lines │ total_units_sold │ total_sales_dollars │
│ varchar  │   int64    │      int64      │      int128      │       double        │
├──────────┼────────────┼─────────────────┼──────────────────┼─────────────────────┤
│ Boating  │   31497118 │              10 │        134187726 │      33882004193.55 │
│ Camping  │   31510602 │              10 │        134243173 │      33896037283.11 │
│ Decoys   │   31477583 │              10 │        134019052 │      33842622818.51 │
│ Optics   │   31490467 │              10 │        134149126 │      33864669775.03 │
│ Footwear │   31527195 │              10 │        134369593 │      33931241721.58 │
└──────────┴────────────┴─────────────────┴──────────────────┴─────────────────────┘



In [2]:
import duckdb
cn = duckdb.connect('~/test_dbs/duckland_local.db')
cn.sql("from v_order_header limit 3").show()
cn.sql("from v_order_detail limit 3").show()
cn.sql("""
    select 'order header' as source, count(*) as cnt from v_order_header
    union all
    select 'order detail' as source, count(*) as cnt from v_order_detail
""").show()
cn.close()

┌──────────┬─────────────┬───────────────────────────────┬─────────┬──────────┬─────────────┬────────────────┬──────────┬─────────────┐
│ order_id │ customer_id │          order_date           │ status  │ store_id │   channel   │ payment_method │ ship_zip │ total_cents │
│  int64   │    int64    │   timestamp with time zone    │ varchar │  int64   │   varchar   │    varchar     │ varchar  │    int64    │
├──────────┼─────────────┼───────────────────────────────┼─────────┼──────────┼─────────────┼────────────────┼──────────┼─────────────┤
│        0 │     8959605 │ 2026-01-14 20:20:31.442449-05 │ NEW     │     4322 │ Marketplace │ Cash           │ 69473    │      770350 │
│        1 │     1307370 │ 2026-01-14 20:20:31.442449-05 │ SHIP    │     2248 │ Online      │ ApplePay       │ 27546    │      672502 │
│        2 │     7676886 │ 2026-01-14 20:20:31.442449-05 │ CLOSE   │     2194 │ Store       │ Card           │ 61862    │      976152 │
└──────────┴─────────────┴──────────────────────

In [7]:
import os
import duckdb
db_name = "duckland"

cn = duckdb.connect(f'md:{db_name}?motherduck_token={os.getenv("MD_TOKEN")}')
cn.execute("attach '~/test_dbs/duckland_local.db' as local_db")

sql = f"""
    EXPLAIN
    create or replace table duckland.main.channel_loc_qty
    as
    select oh.channel
        , iv.location
        , sum(od.qty) as tot_qty
    from local_db.v_order_header oh
        join local_db.v_order_detail as od 
            on oh.order_id = od.order_id
        join duckland.main.inventory as iv 
            on od.sku = iv.sku
    group by all
"""
res = cn.execute(sql)
plan_text = res.fetchall()[0][1]
print(plan_text)

┌───────────────────────────┐
│    DOWNLOAD_SOURCE (L)    │
│    ────────────────────   │
│          md_type:         │
│      DOWNLOAD_SOURCE      │
│                           │
│        bridge_id: 1       │
└─────────────┬─────────────┘
┌─────────────┴─────────────┐
│     DOWNLOAD_SINK (R)     │
│    ────────────────────   │
│          md_type:         │
│       DOWNLOAD_SINK       │
│                           │
│        bridge_id: 1       │
│      parallel: false      │
└─────────────┬─────────────┘
┌─────────────┴─────────────┐
│ BATCH_CREATE_TABLE_AS (R) │
└─────────────┬─────────────┘
┌─────────────┴─────────────┐
│     UPLOAD_SOURCE (R)     │
│    ────────────────────   │
│          md_type:         │
│       UPLOAD_SOURCE       │
│                           │
│        bridge_id: 2       │
└─────────────┬─────────────┘
┌─────────────┴─────────────┐
│      UPLOAD_SINK (L)      │
│    ────────────────────   │
│    md_type: UPLOAD_SINK   │
│        bridge_id: 2       │
│       pa